## Welcome to the Ontology Tutorial hand-on session. 

Here, we will demonstrate how to use ontologies through the OWLAPI, classify ontologies, reason over them, and compute semantic similarity. In the second part, we will also use machine learning with ontologies.

If you have not done so, please download our data package from http://aber-owl.net/aber-owl/diseasephenotypes/ontology/ontology-tutorial.tar.gz

To run a piece of code, just go to the box, and either press Shift+Enter, or click on the "run cell" button in the menu bar.

__Please run the first cell before the tutorial!__
This will download all the Java libraries we need.

Go ahead now and run the first cell. It may take a while to download all the libraries, so you may want to get a coffee while it's running.

In [ ]:
// Download dependencies using Groovy's Grape system
import groovy.grape.Grape
Grape.grab(group:'org.semanticweb.elk', module:'elk-owlapi', version:'0.4.3')
Grape.grab(group:'net.sourceforge.owlapi', module:'owlapi-api', version:'4.2.5')
Grape.grab(group:'net.sourceforge.owlapi', module:'owlapi-apibinding', version:'4.2.5')
Grape.grab(group:'net.sourceforge.owlapi', module:'owlapi-impl', version:'4.2.5')
Grape.grab(group:'com.github.sharispe', module:'slib-sml', version:'0.9.1')

We will now show some basic operations on ontologies. We need to import lots of classes from the OWLAPI, create an OWL ontology manager, load the ontology, and start exploring a bit.

The code below may need to be modified a bit. In the line `ont = manager.loadOntologyFromOntologyDocument(new IRI("file:merged-phenomenet.owl"))`, please change the path to where you have extracted the content of our data package. For example, if you extracted the content of our data package to `/tmp/`, replace `file:merged-phenomenet.owl` with `file:/tmp/merged-phenomenet.owl`.

__If you have memory problems:__ The ontology is rather large and requires around 10GB of memory to load (and later to classify). If you have problems loading this ontology, download `phenomenet-inferred.owl` from our Github site and replace `file:merged-phenomenet.owl` with `file:phenomenet-inferred.owl` in the code below (you just need to comment out the line which loads the ontology and uncomment the alternative).

In [ ]:
// opens ontology file and prints the number of classes

// import what we need
import java.net.*
import org.semanticweb.owlapi.model.parameters.*
import org.semanticweb.elk.owlapi.ElkReasonerFactory;
import org.semanticweb.elk.owlapi.ElkReasonerConfiguration
import org.semanticweb.elk.reasoner.config.*
import org.semanticweb.owlapi.apibinding.OWLManager;
import org.semanticweb.owlapi.reasoner.*
import org.semanticweb.owlapi.reasoner.structural.StructuralReasoner
import org.semanticweb.owlapi.vocab.OWLRDFVocabulary;
import org.semanticweb.owlapi.model.*;
import org.semanticweb.owlapi.io.*;
import org.semanticweb.owlapi.owllink.*;
import org.semanticweb.owlapi.util.*;
import org.semanticweb.owlapi.search.*;
import org.semanticweb.owlapi.manchestersyntax.renderer.*;
import org.semanticweb.owlapi.reasoner.structural.*
 
// Let's load an ontology and output the number of classes
// Create the ontology manager
manager = OWLManager.createOWLOntologyManager()
// create data factory (to create axioms, classes, etc.)
fac = manager.getOWLDataFactory()
// Load the latest version of the PhenomeNET Ontology; for more information about this ontology, see http://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005500

ont = manager.loadOntologyFromOntologyDocument(new IRI("file:merged-phenomenet.owl"))

// Use the next line instead if you have memory problems loading the ontology above (comment out the line above and uncomment this line):
//ont = manager.loadOntologyFromOntologyDocument(new IRI("file:phenomenet-inferred.owl"))

ont.getClassesInSignature(true).size()

## Reasoning with ontologies

Reasoning means we use inference rules that use the axioms within the ontology to extract new knowledge (which may or may not be explicitly stated). The most basic operation an OWL reasoner does is to perform _classification_, i.e., it identifies for each class its sub- and super-classes. OWL reasoners can also be used to query ontologies, and find, for example, sub-classes of named classes or complex class descriptions.

Reasoning in OWL may be very complex. For example, automated reasoning in OWL 2 is 2NEXPTIME-complete, and will therefore not scale well on arbitrary ontologies. Fortunately, many optimized reasoners exist, so the theoretical limitations are not always a barrier in practise. Nevertheless, OWL 2 comes with several sub-langes (OWL 2 EL, QL, and RL) which guarantee more efficient automated reasoning. In particular the OWL 2 EL language profile is widely used in biological and biomedical ontologies.

In the first box below, we will just create a new OWL reasoner object (using the OWL 2 EL reasoner [ELK](https://github.com/liveontologies/elk-reasoner)). In the second box, we will use this reasoner to find all the subclasses of the "Mode of inheritance" class in the Human Phenotype Ontology, and output the classes and their labels.

In [ ]:
// import what we need
import java.net.*
import org.semanticweb.owlapi.model.parameters.*
import org.semanticweb.elk.owlapi.ElkReasonerFactory;
import org.semanticweb.elk.owlapi.ElkReasonerConfiguration
import org.semanticweb.elk.reasoner.config.*
import org.semanticweb.owlapi.apibinding.OWLManager;
import org.semanticweb.owlapi.reasoner.*
import org.semanticweb.owlapi.reasoner.structural.StructuralReasoner
import org.semanticweb.owlapi.vocab.OWLRDFVocabulary;
import org.semanticweb.owlapi.model.*;
import org.semanticweb.owlapi.io.*;
import org.semanticweb.owlapi.owllink.*;
import org.semanticweb.owlapi.util.*;
import org.semanticweb.owlapi.search.*;
import org.semanticweb.owlapi.manchestersyntax.renderer.*;
import org.semanticweb.owlapi.reasoner.structural.*

ConsoleProgressMonitor progressMonitor = new ConsoleProgressMonitor()
OWLReasonerConfiguration config = new SimpleConfiguration(progressMonitor)
ElkReasonerFactory f1 = new ElkReasonerFactory()
reasoner = f1.createReasoner(ont,config)


In [ ]:
// import what we need
import java.net.*
import org.semanticweb.owlapi.model.parameters.*
import org.semanticweb.elk.owlapi.ElkReasonerFactory;
import org.semanticweb.elk.owlapi.ElkReasonerConfiguration
import org.semanticweb.elk.reasoner.config.*
import org.semanticweb.owlapi.apibinding.OWLManager;
import org.semanticweb.owlapi.reasoner.*
import org.semanticweb.owlapi.reasoner.structural.StructuralReasoner
import org.semanticweb.owlapi.vocab.OWLRDFVocabulary;
import org.semanticweb.owlapi.model.*;
import org.semanticweb.owlapi.io.*;
import org.semanticweb.owlapi.owllink.*;
import org.semanticweb.owlapi.util.*;
import org.semanticweb.owlapi.search.*;
import org.semanticweb.owlapi.manchestersyntax.renderer.*;
import org.semanticweb.owlapi.reasoner.structural.*

// get all subclasses of "Mode of inheritance", including all descendant classes (direct set to "false")
reasoner.getSubClasses(fac.getOWLClass(IRI.create("http://purl.obolibrary.org/obo/HP_0000005")), false).getFlattened().each { cl ->
  def ciri = cl.getIRI()
  // now we get the label (rdfs:label) for each of the classes and print to stdout
  EntitySearcher.getAnnotations(cl, ont, fac.getRDFSLabel()).each { a ->
    OWLAnnotationValue val = a.getValue()
    if(val instanceof OWLLiteral) {
      def label = ((OWLLiteral)val).getLiteral()
      println "$label ($ciri)"
    }
  }
}


Next, we will classify the ontology (from our data package) using the ELK reasoner, and store an inferred version of the ontology in the user home directory. The inferred version will be classified, i.e., all implied sub- and super-class relations are inferred and a transitive reduction is performed so that the resulting taxonomy does not contain redundant sub- or super-class relations.

If you do not want to store the file in `$HOME/phenomenet-inferred.owl`, change the code below accordingly.

In [ ]:
// import what we need
import java.net.*
import org.semanticweb.owlapi.model.parameters.*
import org.semanticweb.elk.owlapi.ElkReasonerFactory;
import org.semanticweb.elk.owlapi.ElkReasonerConfiguration
import org.semanticweb.elk.reasoner.config.*
import org.semanticweb.owlapi.apibinding.OWLManager;
import org.semanticweb.owlapi.reasoner.*
import org.semanticweb.owlapi.reasoner.structural.StructuralReasoner
import org.semanticweb.owlapi.vocab.OWLRDFVocabulary;
import org.semanticweb.owlapi.model.*;
import org.semanticweb.owlapi.io.*;
import org.semanticweb.owlapi.owllink.*;
import org.semanticweb.owlapi.util.*;
import org.semanticweb.owlapi.search.*;
import org.semanticweb.owlapi.manchestersyntax.renderer.*;
import org.semanticweb.owlapi.reasoner.structural.*
    
// write the inferred version of the ontology to a file (phenomenet-inferred.owl in the user homedir)
def homeDir = System.getProperty("user.home")
File outfile = new File (homeDir, "phenomenet-inferred.owl")
def outont = manager.createOntology(new IRI("http://aber-owl.net/ismb-tutorial/phenomenet-inferred.owl"))
InferredOntologyGenerator generator = new InferredOntologyGenerator(reasoner, [new InferredSubClassAxiomGenerator()])
generator.fillOntology(fac, outont)

manager.saveOntology(outont, IRI.create("file:"+outfile.getAbsolutePath()))


## Semantic similarity

We use the [Semantic Measures Library](http://semantic-measures-library.org) to compute semantic similarity over the inferred ontology. We first generate some basic objects we need to define semantic similarity, and populate the semantic graph (used by the Semantic Measures Library) with the inferred taxonomy generated above.

Note: while it is possible to compute semantic similarity over the non-inferred version of the ontology, most similarity measures crucially rely on a graph structure that resembles (or approximates) the semantic content of the ontology. Using a non-inferred version will hide this content. We therefore _always_ recommend to apply semantic similarity measures on the fully inferred version of an ontology.

In [ ]:
import org.openrdf.model.vocabulary.*
import slib.sglib.io.loader.*
import slib.sml.sm.core.metrics.ic.utils.*
import slib.sml.sm.core.utils.*
import slib.sglib.io.loader.bio.obo.*
import org.openrdf.model.URI
import slib.graph.algo.extraction.rvf.instances.*
import slib.sglib.algo.graph.utils.*
import slib.utils.impl.Timer
import slib.graph.algo.extraction.utils.*
import slib.graph.model.graph.*
import slib.graph.model.repo.*
import slib.graph.model.impl.graph.memory.*
import slib.sml.sm.core.engine.*
import slib.graph.io.conf.*
import slib.graph.model.impl.graph.elements.*
import slib.graph.algo.extraction.rvf.instances.impl.*
import slib.graph.model.impl.repo.*
import slib.graph.io.util.*
import slib.graph.io.loader.*
    
factory = URIFactoryMemory.getSingleton()
URI graph_uri = factory.getURI("http://aber-owl.net/ismb-tutorial/")
graph = new GraphMemory(graph_uri)


In [ ]:
import org.openrdf.model.vocabulary.*
import slib.sglib.io.loader.*
import slib.sml.sm.core.metrics.ic.utils.*
import slib.sml.sm.core.utils.*
import slib.sglib.io.loader.bio.obo.*
import org.openrdf.model.URI
import slib.graph.algo.extraction.rvf.instances.*
import slib.sglib.algo.graph.utils.*
import slib.utils.impl.Timer
import slib.graph.algo.extraction.utils.*
import slib.graph.model.graph.*
import slib.graph.model.repo.*
import slib.graph.model.impl.graph.memory.*
import slib.sml.sm.core.engine.*
import slib.graph.io.conf.*
import slib.graph.model.impl.graph.elements.*
import slib.graph.algo.extraction.rvf.instances.impl.*
import slib.graph.model.impl.repo.*
import slib.graph.io.util.*
import slib.graph.io.loader.*
    
def homeDir = System.getProperty("user.home")
File ontfile = new File (homeDir, "phenomenet-inferred.owl")
GDataConf graphconf = new GDataConf(GFormat.RDF_XML, ontfile.getCanonicalPath())
GraphLoaderGeneric.populate(graphconf, graph)


Sometimes, ontologies (or the taxonomies underlying them) have more than one root (below `owl:Thing`). For example, the Gene Ontology has three roots, _Biological process_, _Molecular function_, and _Cellular Component_. To make classes comparable when they do not share a root (necessary for example in path-based similarity measures), we generate a new artificial root and make this the new root of our semantic graph.

In [ ]:
import org.openrdf.model.vocabulary.*
import slib.sglib.io.loader.*
import slib.sml.sm.core.metrics.ic.utils.*
import slib.sml.sm.core.utils.*
import slib.sglib.io.loader.bio.obo.*
import org.openrdf.model.URI
import slib.graph.algo.extraction.rvf.instances.*
import slib.sglib.algo.graph.utils.*
import slib.utils.impl.Timer
import slib.graph.algo.extraction.utils.*
import slib.graph.model.graph.*
import slib.graph.model.repo.*
import slib.graph.model.impl.graph.memory.*
import slib.sml.sm.core.engine.*
import slib.graph.io.conf.*
import slib.graph.model.impl.graph.elements.*
import slib.graph.algo.extraction.rvf.instances.impl.*
import slib.graph.model.impl.repo.*
import slib.graph.io.util.*
import slib.graph.io.loader.*


URI virtualRoot = factory.getURI("http://aber-owl.net/ismb-tutorial/virtualRoot")
graph.addV(virtualRoot)
GAction rooting = new GAction(GActionType.REROOTING)
rooting.addParameter("root_uri", virtualRoot.stringValue())
GraphActionExecutor.applyAction(factory, rooting, graph)


Now we configure the semantic similarity measure we want to use.

Because we do not have instances, we use only the ontology structure to determine how specific a class is (`IC_Conf_Topo` below for use of a specificity measure based only on the topology of the semantic graph). We use Resnik's similarity measure ("similarity of two classes is the specificity value of their most specific common ancestor").

In [ ]:
import org.openrdf.model.vocabulary.*
import slib.sglib.io.loader.*
import slib.sml.sm.core.metrics.ic.utils.*
import slib.sml.sm.core.utils.*
import slib.sglib.io.loader.bio.obo.*
import org.openrdf.model.URI
import slib.graph.algo.extraction.rvf.instances.*
import slib.sglib.algo.graph.utils.*
import slib.utils.impl.Timer
import slib.graph.algo.extraction.utils.*
import slib.graph.model.graph.*
import slib.graph.model.repo.*
import slib.graph.model.impl.graph.memory.*
import slib.sml.sm.core.engine.*
import slib.graph.io.conf.*
import slib.graph.model.impl.graph.elements.*
import slib.graph.algo.extraction.rvf.instances.impl.*
import slib.graph.model.impl.repo.*
import slib.graph.io.util.*
import slib.graph.io.loader.*

// configure the semantic similarity measure: use Resnik's (extrinsic) information content measure, and Resnik's similarity measure
icConf = new IC_Conf_Topo("Sanchez", SMConstants.FLAG_ICI_SANCHEZ_2011)
smConfPairwise = new SMconf("Resnik", SMConstants.FLAG_SIM_PAIRWISE_DAG_NODE_RESNIK_1995 )
smConfPairwise.setICconf(icConf)

// initialize the engine
engine = new SM_Engine(graph)



Now we can compare some classes and determine how similar they are. Feel free to change the code to some new classes and see how the similarity changes.

You can use classes from the Human Phenotype Ontology (HPO) and the Mammalian Phenotype Ontology (MPO) for similarity computation between phenotypes. Since we are using the PhenomeNET ontology (see [our paper](http://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005500)), classes in both ontologies are actually comparable.

In [ ]:
// similarity between two classes from HPO
cl1 = factory.getURI("http://purl.obolibrary.org/obo/HP_0030766") // Ear pain
cl2 = factory.getURI("http://purl.obolibrary.org/obo/HP_0012781") // mid frequency hearing loss
println "Similarity between 'Ear pain' and 'mid frequency hearing loss': "+engine.compare(smConfPairwise, cl1, cl2)


In [ ]:
// similarity between two classes from HPO
cl2 = factory.getURI("http://purl.obolibrary.org/obo/HP_0001636") // tetralogy of fallot
println "Similarity between 'Ear pain' and 'tetralogy of fallot': "+engine.compare(smConfPairwise, cl1, cl2)


In [ ]:
// similarity between two classes, one from HPO and one from MP
cl1 = factory.getURI("http://purl.obolibrary.org/obo/MP_0004084") // abnormal cardiac muscle relaxation
cl2 = factory.getURI("http://purl.obolibrary.org/obo/HP_0001636") // tetralogy of fallot
println "Similarity between 'abnormal cardiac muscle relaxation' (MP) and 'tetralogy of fallot' (HP): "+engine.compare(smConfPairwise, cl1, cl2)


In [ ]:
// similarity between two classes, one from HPO and one from MP
cl1 = factory.getURI("http://purl.obolibrary.org/obo/MP_0010402") // ventricular septal defect
println "Similarity between 'ventricular septal defect' (MP) and 'tetralogy of fallot' (HP): "+engine.compare(smConfPairwise, cl1, cl2)


So far, we have only looked at comparisons between two individual classes. This may be nice if we just want to understand how the classes in an ontology are related, but usually we want to compare _things that are annotated with classes_ from an ontology.

We now download mouse phenotypes resulting from single gene knockouts (note: replace `http://www.informatics.jax.org/downloads/reports/MGI_GenePheno.rpt` with a `file:...` URL in case of slow Internet connection; you can find the file `MGI_GenePheno.rpt` in our data package), and add each gene as an instance of its phenotype classes (using an `rdf:type` edge). 

Important: this is __not__ the right way to do this in an ontology but is an artifact of the Semantic Measures Library. When building an ontology, a _gene_ is not an _instance_ of a phenotype, but will be related to the phenotype through other kinds of relations. In the Semantic Measures Library, we will treat them as instances, however.

In [ ]:
import org.openrdf.model.vocabulary.*
import slib.sglib.io.loader.*
import slib.sml.sm.core.metrics.ic.utils.*
import slib.sml.sm.core.utils.*
import slib.sglib.io.loader.bio.obo.*
import org.openrdf.model.URI
import slib.graph.algo.extraction.rvf.instances.*
import slib.sglib.algo.graph.utils.*
import slib.utils.impl.Timer
import slib.graph.algo.extraction.utils.*
import slib.graph.model.graph.*
import slib.graph.model.repo.*
import slib.graph.model.impl.graph.memory.*
import slib.sml.sm.core.engine.*
import slib.graph.io.conf.*
import slib.graph.model.impl.graph.elements.*
import slib.graph.algo.extraction.rvf.instances.impl.*
import slib.graph.model.impl.repo.*
import slib.graph.io.util.*
import slib.graph.io.loader.*

// now we download mouse phenotype annotations and add them to our graph as instances (using rdf:type)
new URL ("file:MGI_GenePheno.rpt").getText().splitEachLine("\t") { line ->
  def geneid = line[6]
  def idUri = factory.getURI("http://phenomebrowser.net/ismb-tutorial/gene/"+geneid)
  def pheno = line[4].replaceAll(":","_")
  def phenoUri = factory.getURI("http://purl.obolibrary.org/obo/"+pheno)
  Edge e = new Edge(idUri, RDF.TYPE, phenoUri)
  graph.addE(e)
}


In the presence of instances, we can change our class specificity measure. We now use Resnik's specificity measure, which is just the information content of a class (i.e., `-log(p(c))` where `p(c)` is the probability to have an instance annotated with the class. This is a specificity measure that relies on instances, and is therefore a _corpus-based_ measure (using `IC_Conf_Corpus` below).

We also configure a group-wise measure now, using the Best-Matching Average strategy. This will allow us to compare sets of classes to sets of classes.

Finally, the `InstanceAccessor` will allow us to identify all instances of our classes.

In [ ]:
import org.openrdf.model.vocabulary.*
import slib.sglib.io.loader.*
import slib.sml.sm.core.metrics.ic.utils.*
import slib.sml.sm.core.utils.*
import slib.sglib.io.loader.bio.obo.*
import org.openrdf.model.URI
import slib.graph.algo.extraction.rvf.instances.*
import slib.sglib.algo.graph.utils.*
import slib.utils.impl.Timer
import slib.graph.algo.extraction.utils.*
import slib.graph.model.graph.*
import slib.graph.model.repo.*
import slib.graph.model.impl.graph.memory.*
import slib.sml.sm.core.engine.*
import slib.graph.io.conf.*
import slib.graph.model.impl.graph.elements.*
import slib.graph.algo.extraction.rvf.instances.impl.*
import slib.graph.model.impl.repo.*
import slib.graph.io.util.*
import slib.graph.io.loader.*


// now that we have instances/annotations, we switch to Resnik's information content measure
icConf = new IC_Conf_Corpus("ResnikIC", SMConstants.FLAG_IC_ANNOT_RESNIK_1995_NORMALIZED)
smConfPairwise.setICconf(icConf)
// using Best-Matching Average to merge annotations when comparing sets of classes
smConfGroupwise = new SMconf("BMA", SMConstants.FLAG_SIM_GROUPWISE_BMA)
// and re-initialize the graph
engine = new SM_Engine(graph)

// we need this to find our instances again
ia = new InstanceAccessor_RDF_TYPE(graph)



We can now compute again similarity between two classes. Note how the similarity values change. Feel free to experiment with this and compare different classes.

In [ ]:
// similarity between two classes, one from HPO and one from MP
cl1 = factory.getURI("http://purl.obolibrary.org/obo/MP_0004084") // abnormal cardiac muscle relaxation
cl2 = factory.getURI("http://purl.obolibrary.org/obo/HP_0001636") // tetralogy of fallot
println "Similarity (Resnik) between 'abnormal cardiac muscle relaxation' (MP) and 'tetralogy of fallot' (HP): "+engine.compare(smConfPairwise, cl1, cl2)


In [ ]:
// similarity between two classes, one from HPO and one from MP
cl1 = factory.getURI("http://purl.obolibrary.org/obo/MP_0010402") // ventricular septal defect
println "Similarity (Resnik) between 'ventricular septal defect' (MP) and 'tetralogy of fallot' (HP): "+engine.compare(smConfPairwise, cl1, cl2)


With the presence of mouse genes as instances in our semantic graph, we can do some more sophisticated analyses. Here, we define a patient through a set of phenotypes (i.e., clinical signs and symptoms). The patient has a rare, heritable form of macular dystrophy without known etiology (see https://www.omim.org/entry/153840). We hypothesize that there may be a mouse knockout that resembles this disease, and possibly the human ortholog of the mouse gene may also be involved in this form of macular dystrophy.

We compare our patient to _all_ instances in our graph (i.e., all mouse knockouts) using our measure of phenotypic similarity. We then sort the resulting list by their similarity score in descending order, and output the top 10 predictions.

In [ ]:
// that's our patient, with a single phenotype of an orphan disease, "MACULAR DYSTROPHY, VITELLIFORM, 1; VMD1" (https://www.omim.org/entry/153840)
Set patient = [ factory.getURI("http://purl.obolibrary.org/obo/HP_0007754"), factory.getURI("http://purl.obolibrary.org/obo/HP_0007663"), factory.getURI("http://purl.obolibrary.org/obo/HP_0001123"), 
  factory.getURI("http://purl.obolibrary.org/obo/HP_0000505"), factory.getURI("http://purl.obolibrary.org/obo/HP_0007677") ]
// we store the similarity values in the results set
def results = new LinkedHashSet()
engine.getInstances().each { gene ->
  def phenoSet = ia.getDirectClass(gene) // gets all the annotations of each instance (gene)
  Expando exp = new Expando()
  try { // this part might fail if the version of MP we use to compute similarity and the MP used for annotation is different; we ignore all errors for now
    exp.sim = engine.compare(smConfGroupwise, smConfPairwise, phenoSet, patient)
    exp.gene = gene
    results.add(exp)
  } catch (Exception E) {}
}
// sorting results by similarity (highest first) and output the top 10 predictions
println "Highest ranking genes for patient 1 (macular dystrophy): "+results.sort { it.sim }.reverse()[0..10]


We test this with another patient, this time with Wiskott-Aldrich Syndrome (https://www.omim.org/entry/301000). 
Wiskott-Aldrich Syndrome is a rare genetically-based disease in which the WAS gene is known to be causally involved. In the mouse, there is also a FOXP3 model that is used as a disease model for Wiskott-Aldrich syndrome (http://www.informatics.jax.org/allele/MGI:1857034).

In [ ]:
// another patient with Wiskott-Aldrich Syndrome
Set patient = [ factory.getURI("http://purl.obolibrary.org/obo/HP_0000112"), factory.getURI("http://purl.obolibrary.org/obo/HP_0000225"), factory.getURI("http://purl.obolibrary.org/obo/HP_0000246"), factory.getURI("http://purl.obolibrary.org/obo/HP_0000388"), factory.getURI("http://purl.obolibrary.org/obo/HP_0000421"), factory.getURI("http://purl.obolibrary.org/obo/HP_0000964"), factory.getURI("http://purl.obolibrary.org/obo/HP_0000967"), factory.getURI("http://purl.obolibrary.org/obo/HP_0000979"), factory.getURI("http://purl.obolibrary.org/obo/HP_0001287"), factory.getURI("http://purl.obolibrary.org/obo/HP_0001419"), factory.getURI("http://purl.obolibrary.org/obo/HP_0001873"), factory.getURI("http://purl.obolibrary.org/obo/HP_0001878"), factory.getURI("http://purl.obolibrary.org/obo/HP_0001888"), factory.getURI("http://purl.obolibrary.org/obo/HP_0001891"), factory.getURI("http://purl.obolibrary.org/obo/HP_0001983"), factory.getURI("http://purl.obolibrary.org/obo/HP_0002037"), factory.getURI("http://purl.obolibrary.org/obo/HP_0002090"), factory.getURI("http://purl.obolibrary.org/obo/HP_0002248"), factory.getURI("http://purl.obolibrary.org/obo/HP_0002249"), factory.getURI("http://purl.obolibrary.org/obo/HP_0002783"), factory.getURI("http://purl.obolibrary.org/obo/HP_0002788"), factory.getURI("http://purl.obolibrary.org/obo/HP_0002848"), factory.getURI("http://purl.obolibrary.org/obo/HP_0002850"), factory.getURI("http://purl.obolibrary.org/obo/HP_0002963"), factory.getURI("http://purl.obolibrary.org/obo/HP_0002971"), factory.getURI("http://purl.obolibrary.org/obo/HP_0003010"), factory.getURI("http://purl.obolibrary.org/obo/HP_0003212"), factory.getURI("http://purl.obolibrary.org/obo/HP_0003261"), factory.getURI("http://purl.obolibrary.org/obo/HP_0005310"), factory.getURI("http://purl.obolibrary.org/obo/HP_0005537"), factory.getURI("http://purl.obolibrary.org/obo/HP_0011944"), factory.getURI("http://purl.obolibrary.org/obo/HP_0040184") ]
def results = new LinkedHashSet()
engine.getInstances().each { gene ->
  def phenoSet = ia.getDirectClass(gene) // gets all the annotations of each instance (gene)
  Expando exp = new Expando()
  try { // this might fail if the version of MP we use to compute similarity and the MP used for annotation is different; we ignore this for now
    exp.sim = engine.compare(smConfGroupwise, smConfPairwise, phenoSet, patient)
    exp.gene = gene
    results.add(exp)
  } catch (Exception E) { }
}
// sorting results by similarity (highest first) and output the top 10 predictions
println "Highest ranking genes for patient 2 (Wiskott Aldrich Syndrom): "+results.sort { it.sim }.reverse()[0..10] // includes MGI:1891436, FOXP3
